In [1]:
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
import json
import time


def hash_value(data):
    h = SHA256.new()
    h.update(data)
    return h.hexdigest()

In [2]:
sent_messages = []
smart_meter_ids = ["SM1", "SM2", "SM3"]
serial_numbers = ["1234567890", "9876543210", "1357924680"]
gateway_key = b"abcdefghijklmnop"
registration_info = {
    "SM1": {
        "h_sn": hash_value("1234567890".encode()),
        "n_i": hash_value(("SM1" + hash_value("1234567890".encode())).encode())
        + hash_value(gateway_key + b"x_i_1"),
        "x_i": b"x_i_1".hex(),
    },
    "SM2": {
        "h_sn": hash_value("9876543210".encode()),
        "n_i": hash_value(("SM2" + hash_value("9876543210".encode())).encode())
        + hash_value(gateway_key + b"x_i_2"),
        "x_i": b"x_i_2".hex(),
    },
    "SM3": {
        "h_sn": hash_value("1357924680".encode()),
        "n_i": hash_value(("SM3" + hash_value("1357924680".encode())).encode())
        + hash_value(gateway_key + b"x_i_3"),
        "x_i": b"x_i_3".hex(),
    },
}
session_keys = {}

In [3]:
def generate_random(size):
    return get_random_bytes(size)


def get_timestamp():
    return int(time.time())


def send_message(message):
    json_message = json.dumps(message)
    print("Sending message:", json_message)
    sent_messages.append(message)


def smart_meter_registration(smart_meter_id, serial_number):
    h_sn = hash_value(serial_number.encode())
    message = {"smart_meter_id": smart_meter_id, "h_sn": h_sn}
    send_message(message)


def gateway_registration_processing(message, gateway_key):
    smart_meter_id = message["smart_meter_id"]
    h_sn = message["h_sn"]
    x_i = generate_random(16)
    n_i = hash_value((smart_meter_id + h_sn).encode()) + hash_value(gateway_key + x_i)
    registration_info = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i,
        "x_i": x_i.hex(),
    }
    print("Storing registration information:", registration_info)
    response_message = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i,
        "x_i": x_i.hex(),
    }
    send_message(response_message)


def smart_meter_authentication(smart_meter_id, h_sn, n_i, x_i, gateway_key):
    timestamp = get_timestamp()
    did_i = hash_value((smart_meter_id + h_sn + x_i.hex() + str(timestamp)).encode())
    k_i = generate_random(16)
    pk_i = (
        hash_value((gateway_key + x_i).hex().encode() + str(timestamp).encode())
        + k_i.hex()
    )
    b_i = hash_value((did_i + x_i.hex() + str(timestamp) + pk_i).encode())
    message = {
        "smart_meter_id": smart_meter_id,
        "did_i": did_i,
        "b_i": b_i,
        "timestamp": timestamp,
        "pk_i": pk_i,
    }
    send_message(message)


def gateway_authentication(message, gateway_key, registration_info):
    smart_meter_id = message["smart_meter_id"]
    did_i = message["did_i"]
    b_i = message["b_i"]
    timestamp = message["timestamp"]
    pk_i = message["pk_i"]
    h_sn = registration_info[smart_meter_id]["h_sn"]
    n_i = registration_info[smart_meter_id]["n_i"]
    x_i = bytes.fromhex(registration_info[smart_meter_id]["x_i"])
    current_timestamp = get_timestamp()
    if abs(current_timestamp - timestamp) > 300:
        print("Timestamp verification failed.")
        return
    expected_did_i = hash_value(
        (smart_meter_id + h_sn + x_i.hex() + str(timestamp)).encode()
    )
    if did_i != expected_did_i:
        print("Dynamic identity verification failed.")
        return
    expected_b_i = hash_value((did_i + x_i.hex() + str(timestamp) + pk_i).encode())
    if b_i != expected_b_i:
        print("B_i verification failed.")
        return
    k_i = bytes.fromhex(pk_i[64:])
    k_gw = generate_random(16)
    pk_gw = k_gw.hex() + hash_value(k_i + str(timestamp).encode() + x_i)
    c_i = hash_value(
        did_i.encode() + str(timestamp).encode() + k_i.hex().encode() + pk_gw.encode()
    ) + hash_value(gateway_key + x_i)
    response_message = {
        "smart_meter_id": smart_meter_id,
        "c_i": c_i,
        "timestamp": timestamp,
        "pk_gw": pk_gw,
    }
    send_message(response_message)
    session_key = hash_value(k_i + k_gw)
    print("Shared session key for", smart_meter_id + ":", session_key)


def key_refreshment(smart_meter_id, session_key, gateway_key, x_i):
    new_session_key = hash_value(
        session_key.encode() + hash_value((gateway_key + x_i)).encode()
    )
    print("New session key for", smart_meter_id + ":", new_session_key)
    smart_meter_authentication(
        smart_meter_id,
        registration_info[smart_meter_id]["h_sn"],
        registration_info[smart_meter_id]["n_i"],
        bytes.fromhex(registration_info[smart_meter_id]["x_i"]),
        gateway_key,
    )


def establish_multicast_group(group_id, smart_meter_ids, gateway_key):
    group_key = generate_random(16)
    for smart_meter_id in smart_meter_ids:
        x_a = generate_random(16)
        session_key = bytes.fromhex(session_keys[smart_meter_id])
        cipher = AES.new(session_key, AES.MODE_EAX)
        nonce = cipher.nonce
        ciphertext, tag = cipher.encrypt_and_digest(
            json.dumps(
                {
                    "group_id": group_id,
                    "timestamp": get_timestamp(),
                    "x_a": x_a.hex(),
                    "group_key": group_key.hex(),
                }
            ).encode()
        )
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": (nonce + tag + ciphertext).hex(),
        }
        send_message(message)


def join_multicast_group(message, gateway_key):
    smart_meter_id = message["smart_meter_id"]
    timestamp = message["timestamp"]
    encrypted_data = bytes.fromhex(message["encrypted_data"])
    nonce = encrypted_data[:16]
    tag = encrypted_data[16:32]
    ciphertext = encrypted_data[32:]
    session_key = bytes.fromhex(session_keys[smart_meter_id])
    cipher = AES.new(session_key, AES.MODE_EAX, nonce=nonce)
    try:
        plaintext = cipher.decrypt_and_verify(ciphertext, tag)
        multicast_info = json.loads(plaintext.decode())
        group_id = multicast_info["group_id"]
        x_a = bytes.fromhex(multicast_info["x_a"])
        group_key = bytes.fromhex(multicast_info["group_key"])
        response_message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": ciphertext.hex(),
        }
        send_message(response_message)
        multicast_groups[group_id] = {
            "group_key": group_key,
            "members": [smart_meter_id],
        }
        print("Smart meter", smart_meter_id, "joined multicast group", group_id)
    except (ValueError, KeyError) as e:
        print(
            "Error occurred during multicast group join for smart meter", smart_meter_id
        )
        print("Error message:", str(e))


for i in range(len(smart_meter_ids)):
    smart_meter_registration(smart_meter_ids[i], serial_numbers[i])
for i in range(len(smart_meter_ids)):
    registration_request = {
        "smart_meter_id": smart_meter_ids[i],
        "h_sn": hash_value(serial_numbers[i].encode()),
    }
    gateway_registration_processing(registration_request, gateway_key)
for i in range(len(smart_meter_ids)):
    smart_meter_authentication(
        smart_meter_ids[i],
        registration_info[smart_meter_ids[i]]["h_sn"],
        registration_info[smart_meter_ids[i]]["n_i"],
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
        gateway_key,
    )
    session_keys[smart_meter_ids[i]] = hash_value(
        generate_random(16) + generate_random(16)
    )
for i in range(len(smart_meter_ids)):
    message = {
        "smart_meter_id": smart_meter_ids[i],
        "did_i": hash_value(
            (
                smart_meter_ids[i]
                + registration_info[smart_meter_ids[i]]["h_sn"]
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
            ).encode()
        ),
        "b_i": hash_value(
            (
                hash_value(
                    (
                        smart_meter_ids[i]
                        + registration_info[smart_meter_ids[i]]["h_sn"]
                        + registration_info[smart_meter_ids[i]]["x_i"]
                        + str(get_timestamp())
                    ).encode()
                )
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
                + session_keys[smart_meter_ids[i]]
            ).encode()
        ),
        "timestamp": get_timestamp(),
        "pk_i": session_keys[smart_meter_ids[i]],
    }
    gateway_authentication(message, gateway_key, registration_info)
for i in range(len(smart_meter_ids)):
    key_refreshment(
        smart_meter_ids[i],
        session_keys[smart_meter_ids[i]],
        gateway_key,
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
    )
multicast_groups = {}

group_id = "GROUP1"
establish_multicast_group(group_id, ["SM1", "SM2"], gateway_key)
for smart_meter_id in ["SM1", "SM2"]:
    encrypted_data = None
    for message in sent_messages:
        if message["smart_meter_id"] == smart_meter_id:
            if "encrypted_data" in message:
                encrypted_data = message["encrypted_data"]
                break
    if encrypted_data:
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": encrypted_data,
        }
        join_multicast_group(message, gateway_key)

Sending message: {"smart_meter_id": "SM1", "h_sn": "c775e7b757ede630cd0aa1113bd102661ab38829ca52a6422ab782862f268646"}
Sending message: {"smart_meter_id": "SM2", "h_sn": "7619ee8cea49187f309616e30ecf54be072259b43760f1f550a644945d5572f2"}
Sending message: {"smart_meter_id": "SM3", "h_sn": "0139ac6fa1fb1ec90bc15fe5eb13421f32579bce849ab697aa3ef3b77823ae17"}
Storing registration information: {'smart_meter_id': 'SM1', 'n_i': 'b7b8278e678ebf3e18a333f22a8bbb840851278a24f2dc91a9cf8beb3142edf3c192dbcfdebc492d8fe2fc512ffbb594c98fdac83c28312601b1e3804ddfa0f4', 'x_i': 'ff9769224b881d5275286ca85a167f68'}
Sending message: {"smart_meter_id": "SM1", "n_i": "b7b8278e678ebf3e18a333f22a8bbb840851278a24f2dc91a9cf8beb3142edf3c192dbcfdebc492d8fe2fc512ffbb594c98fdac83c28312601b1e3804ddfa0f4", "x_i": "ff9769224b881d5275286ca85a167f68"}
Storing registration information: {'smart_meter_id': 'SM2', 'n_i': '6d9280b46fa2ad9a895f9ea7740385e6c35d999d366306fb80cea78784ddb82fa631bff91da0b5ef36b6182d128a3ae01d926cf73061

In [5]:
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
import json


# Hash function
def hash_value(data):
    h = SHA256.new()
    h.update(data)
    return h.hexdigest()


# Generate random value
def generate_random(size):
    return get_random_bytes(size)


# Simulate communication between smart meter and gateway
def send_message(message):
    # Serialize message to JSON
    json_message = json.dumps(message)
    # Simulate sending the message over the network
    # In a real implementation, you would use a communication protocol like MQTT or HTTP
    print("Sending message:", json_message)


# Smart meter registration
def smart_meter_registration(smart_meter_id, serial_number):
    # Compute hash of serial number
    h_sn = hash_value(serial_number.encode())

    # Prepare registration message
    message = {"smart_meter_id": smart_meter_id, "h_sn": h_sn}

    # Send registration message to gateway
    send_message(message)


# Gateway registration processing
def gateway_registration_processing(message, gateway_key):
    # Parse the received message
    smart_meter_id = message["smart_meter_id"]
    h_sn = message["h_sn"]

    # Generate random value for X_i
    x_i = generate_random(16)

    # Compute N_i
    n_i = hash_value((smart_meter_id + h_sn).encode()) + hash_value(gateway_key + x_i)

    # Compute h(h(SN_i))
    h_h_sn = hash_value(hash_value(serial_number.encode()).encode())

    # Store the registration information securely
    # In a real implementation, you would store this in a secure database
    registration_info = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i,
        "h_h_sn": h_h_sn,
        "x_i": x_i.hex(),
    }
    print("Storing registration information:", registration_info)

    # Prepare response message
    response_message = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i,
        "h_h_sn": h_h_sn,
        "x_i": x_i.hex(),
    }

    # Send response message to smart meter
    send_message(response_message)


# Example usage
smart_meter_ids = ["SM1", "SM2", "SM3"]
serial_numbers = ["1234567890", "9876543210", "1357924680"]
gateway_key = b"abcdefghijklmnop"  # Example gateway key

# Smart meters send registration requests
for i in range(len(smart_meter_ids)):
    smart_meter_registration(smart_meter_ids[i], serial_numbers[i])

# Gateway processes the registration requests
for i in range(len(smart_meter_ids)):
    registration_request = {
        "smart_meter_id": smart_meter_ids[i],
        "h_sn": hash_value(serial_numbers[i].encode()),
    }
    gateway_registration_processing(registration_request, gateway_key)

Sending message: {"smart_meter_id": "SM1", "h_sn": "c775e7b757ede630cd0aa1113bd102661ab38829ca52a6422ab782862f268646"}
Sending message: {"smart_meter_id": "SM2", "h_sn": "7619ee8cea49187f309616e30ecf54be072259b43760f1f550a644945d5572f2"}
Sending message: {"smart_meter_id": "SM3", "h_sn": "0139ac6fa1fb1ec90bc15fe5eb13421f32579bce849ab697aa3ef3b77823ae17"}


NameError: name 'serial_number' is not defined

In [ ]:
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
import json
import time


# Hash function
def hash_value(data):
    h = SHA256.new()
    h.update(data)
    return h.hexdigest()


# Generate random value
def generate_random(size):
    return get_random_bytes(size)


# Get current timestamp
def get_timestamp():
    return int(time.time())


# Simulate communication between smart meter and gateway
def send_message(message):
    # Serialize message to JSON
    json_message = json.dumps(message)
    # Simulate sending the message over the network
    # In a real implementation, you would use a communication protocol like MQTT or HTTP
    print("Sending message:", json_message)


# Smart meter authentication and key agreement
def smart_meter_authentication(smart_meter_id, h_sn, n_i, x_i, gateway_key):
    # Generate timestamp
    timestamp = get_timestamp()

    # Compute dynamic identity
    did_i = hash_value((smart_meter_id + h_sn + x_i.hex() + str(timestamp)).encode())

    # Generate random value for K_i
    k_i = generate_random(16)

    # Compute PK_i
    pk_i = (
        hash_value((gateway_key + x_i).hex().encode() + str(timestamp).encode())
        + k_i.hex()
    )

    # Compute B_i
    b_i = hash_value((did_i + x_i.hex() + str(timestamp) + pk_i).encode())

    # Prepare authentication message
    message = {
        "smart_meter_id": smart_meter_id,
        "did_i": did_i,
        "b_i": b_i,
        "timestamp": timestamp,
        "pk_i": pk_i,
    }

    # Send authentication message to gateway
    send_message(message)


# Gateway authentication and key agreement
def gateway_authentication(message, gateway_key, registration_info):
    # Parse the received message
    smart_meter_id = message["smart_meter_id"]
    did_i = message["did_i"]
    b_i = message["b_i"]
    timestamp = message["timestamp"]
    pk_i = message["pk_i"]

    # Retrieve the registration information for the smart meter
    h_sn = registration_info[smart_meter_id]["h_sn"]
    n_i = registration_info[smart_meter_id]["n_i"]
    x_i = bytes.fromhex(registration_info[smart_meter_id]["x_i"])

    # Verify the timestamp
    current_timestamp = get_timestamp()
    if abs(current_timestamp - timestamp) > 300:  # 5 minutes
        print("Timestamp verification failed.")
        return

    # Compute the expected dynamic identity
    expected_did_i = hash_value(
        (smart_meter_id + h_sn + x_i.hex() + str(timestamp)).encode()
    )

    # Verify the dynamic identity
    if did_i != expected_did_i:
        print("Dynamic identity verification failed.")
        return

    # Compute the expected B_i
    expected_b_i = hash_value((did_i + x_i.hex() + str(timestamp) + pk_i).encode())

    # Verify B_i
    if b_i != expected_b_i:
        print("B_i verification failed.")
        return

    # Compute K_i
    k_i = bytes.fromhex(pk_i[64:])

    # Generate random value for K_GW
    k_gw = generate_random(16)

    # Compute PK_GW
    pk_gw = k_gw.hex() + hash_value(k_i + str(timestamp).encode() + x_i)

    # Compute C_i
    c_i = hash_value(
        did_i.encode() + str(timestamp).encode() + k_i.hex().encode() + pk_gw.encode()
    ) + hash_value(gateway_key + x_i)

    # Prepare response message
    response_message = {
        "smart_meter_id": smart_meter_id,
        "c_i": c_i,
        "timestamp": timestamp,
        "pk_gw": pk_gw,
    }

    # Send response message to smart meter
    send_message(response_message)

    # Compute the shared session key
    session_key = hash_value(k_i + k_gw)
    print("Shared session key for", smart_meter_id + ":", session_key)


# Example usage
smart_meter_ids = ["SM1", "SM2", "SM3"]
serial_numbers = ["1234567890", "9876543210", "1357924680"]
gateway_key = b"abcdefghijklmnop"  # Example gateway key

# Registration information stored by the gateway
registration_info = {
    "SM1": {
        "h_sn": hash_value("1234567890".encode()),
        "n_i": hash_value(("SM1" + hash_value("1234567890".encode())).encode())
        + hash_value(gateway_key + b"x_i_1"),
        "x_i": b"x_i_1".hex(),
    },
    "SM2": {
        "h_sn": hash_value("9876543210".encode()),
        "n_i": hash_value(("SM2" + hash_value("9876543210".encode())).encode())
        + hash_value(gateway_key + b"x_i_2"),
        "x_i": b"x_i_2".hex(),
    },
    "SM3": {
        "h_sn": hash_value("1357924680".encode()),
        "n_i": hash_value(("SM3" + hash_value("1357924680".encode())).encode())
        + hash_value(gateway_key + b"x_i_3"),
        "x_i": b"x_i_3".hex(),
    },
}

# Smart meters perform authentication and key agreement
for i in range(len(smart_meter_ids)):
    smart_meter_authentication(
        smart_meter_ids[i],
        registration_info[smart_meter_ids[i]]["h_sn"],
        registration_info[smart_meter_ids[i]]["n_i"],
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
        gateway_key,
    )

# Gateway processes the authentication and key agreement messages
for i in range(len(smart_meter_ids)):
    message = {
        "smart_meter_id": smart_meter_ids[i],
        "did_i": hash_value(
            (
                smart_meter_ids[i]
                + registration_info[smart_meter_ids[i]]["h_sn"]
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
            ).encode()
        ),
        "b_i": hash_value(
            (
                hash_value(
                    (
                        smart_meter_ids[i]
                        + registration_info[smart_meter_ids[i]]["h_sn"]
                        + registration_info[smart_meter_ids[i]]["x_i"]
                        + str(get_timestamp())
                    ).encode()
                )
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
                + "pk_i_example"
            ).encode()
        ),
        "timestamp": get_timestamp(),
        "pk_i": "pk_i_example",
    }
    gateway_authentication(message, gateway_key, registration_info)

Sending message: {"smart_meter_id": "SM1", "did_i": "e236db7212410c7b98f3bff100af281f1b9018906da491a6ee753487abb8d761", "b_i": "fab849ad497b65f7202b88e19788fa6053b7802645f53f8c4851fe311bf1d65f", "timestamp": 1714201172, "pk_i": "b120223312d46f644c9c6b028cefdae1c7a0878a84e43e96139c1ad4abd88be4aab1a1d2bfe4a0de9988e463e9e39c5e"}
Sending message: {"smart_meter_id": "SM2", "did_i": "cff737d4c9e505b08eac8c2db3a3a113150095ab8a7e45f694052c15d75b8626", "b_i": "90296c4c61a79206c2c62b11ff1c02e5f35a9cee8a31a56851cf7e86cd1fbfd6", "timestamp": 1714201172, "pk_i": "8604d4f31d4fc7747a11e282bd4ddd5c247582e2fc26a8fe9352f34dd23c8b501952745e708a0c8e615e0c1060658b3a"}
Sending message: {"smart_meter_id": "SM3", "did_i": "d3b770f27028e4aa572b97eaf34a7fb2dcafc99125fbd7385be70203f975c690", "b_i": "b349c89dc09248d8d2832e83070cf1a09e02cc774521c70f309f8ed93f64a645", "timestamp": 1714201172, "pk_i": "b5c9281b1ea9845661b3d0d64f1cb4ac1d7f8dc87eadf2e44a776cdddda7ebeaf02f961e8d6999b4737498fc6445a958"}
Sending message:

In [ ]:
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
import json
import time

# ... (Previous code remains the same) ...


# Key refreshment
def key_refreshment(smart_meter_id, session_key, gateway_key, x_i):
    # Short-term key refreshment
    new_session_key = hash_value(
        session_key.encode() + hash_value((gateway_key + x_i)).encode()
    )
    print("New session key for", smart_meter_id + ":", new_session_key)

    # Long-term key refreshment
    # Perform authentication and key agreement phase again
    smart_meter_authentication(
        smart_meter_id,
        registration_info[smart_meter_id]["h_sn"],
        registration_info[smart_meter_id]["n_i"],
        bytes.fromhex(registration_info[smart_meter_id]["x_i"]),
        gateway_key,
    )


# Example usage
smart_meter_ids = ["SM1", "SM2", "SM3"]
serial_numbers = ["1234567890", "9876543210", "1357924680"]
gateway_key = b"abcdefghijklmnop"  # Example gateway key

# Registration information stored by the gateway
registration_info = {
    "SM1": {
        "h_sn": hash_value("1234567890".encode()),
        "n_i": hash_value(("SM1" + hash_value("1234567890".encode())).encode())
        + hash_value(gateway_key + b"x_i_1"),
        "x_i": b"x_i_1".hex(),
    },
    "SM2": {
        "h_sn": hash_value("9876543210".encode()),
        "n_i": hash_value(("SM2" + hash_value("9876543210".encode())).encode())
        + hash_value(gateway_key + b"x_i_2"),
        "x_i": b"x_i_2".hex(),
    },
    "SM3": {
        "h_sn": hash_value("1357924680".encode()),
        "n_i": hash_value(("SM3" + hash_value("1357924680".encode())).encode())
        + hash_value(gateway_key + b"x_i_3"),
        "x_i": b"x_i_3".hex(),
    },
}

# Smart meters perform authentication and key agreement
session_keys = {}
for i in range(len(smart_meter_ids)):
    smart_meter_authentication(
        smart_meter_ids[i],
        registration_info[smart_meter_ids[i]]["h_sn"],
        registration_info[smart_meter_ids[i]]["n_i"],
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
        gateway_key,
    )
    # Store the established session key for each smart meter
    session_keys[smart_meter_ids[i]] = hash_value(
        generate_random(16) + generate_random(16)
    )

# Gateway processes the authentication and key agreement messages
for i in range(len(smart_meter_ids)):
    message = {
        "smart_meter_id": smart_meter_ids[i],
        "did_i": hash_value(
            (
                smart_meter_ids[i]
                + registration_info[smart_meter_ids[i]]["h_sn"]
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
            ).encode()
        ),
        "b_i": hash_value(
            (
                hash_value(
                    (
                        smart_meter_ids[i]
                        + registration_info[smart_meter_ids[i]]["h_sn"]
                        + registration_info[smart_meter_ids[i]]["x_i"]
                        + str(get_timestamp())
                    ).encode()
                )
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
                + session_keys[smart_meter_ids[i]]
            ).encode()
        ),
        "timestamp": get_timestamp(),
        "pk_i": session_keys[smart_meter_ids[i]],
    }
    gateway_authentication(message, gateway_key, registration_info)

# Key refreshment for each smart meter
for i in range(len(smart_meter_ids)):
    key_refreshment(
        smart_meter_ids[i],
        session_keys[smart_meter_ids[i]],
        gateway_key,
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
    )

Sending message: {"smart_meter_id": "SM1", "did_i": "882b24c971ea2823833fadaddaf15dc47aae1a262c8915eaafea954a581e1c8f", "b_i": "e7eabf9c28d3850e7d6f4ee9b74b6d9774f6df51b0646b6fb50a6b61b9517a80", "timestamp": 1714201173, "pk_i": "29bfd6250fca532d997882296adfa22b931798caa5db5000a137c0c2629e7b72387f25f421e2d64dfa0f77b566e896b2"}
Sending message: {"smart_meter_id": "SM2", "did_i": "2a0ad30d5f7f877fb226249e4482bdb86b3cf82e202e815d786b6e49277854fb", "b_i": "93f09e94550a54add6b44c7683c4959d1baa340a80c3f6a2ee7def3a21caecba", "timestamp": 1714201173, "pk_i": "b8a2de86a8d33b1a5ca874eab45814af22e39bc667d46d3784086671a2151dd38fc36168aa0780aaa0399f6a02c4cbbc"}
Sending message: {"smart_meter_id": "SM3", "did_i": "4bd7702b118b51cbe59f92f66a97a7183eb6926996e0be12cff12c378acd05f3", "b_i": "40423ebe99775db577cef876d3248308052cab05fa6dcad73746e0dbd99946bb", "timestamp": 1714201173, "pk_i": "b56cb21c4206e237e19ae681ab8067c8efee648340c938f704fe11abfb98bcd53b025504b1665b3e0b2f281419961f17"}
Sending message:

In [ ]:
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
import json
import time

# ... (Previous code remains the same) ...


def send_message(message):
    # Serialize message to JSON
    json_message = json.dumps(message)
    # Simulate sending the message over the network
    # In a real implementation, you would use a communication protocol like MQTT or HTTP
    print("Sending message:", json_message)
    # Store the sent message
    sent_messages.append(message)


# Multicast key generation
def establish_multicast_group(group_id, smart_meter_ids, gateway_key):
    # Generate multicast group key
    group_key = generate_random(16)

    # Prepare and send message to each smart meter in the group
    for smart_meter_id in smart_meter_ids:
        # Generate unique secret value for the smart meter
        x_a = generate_random(16)

        # Encrypt the multicast group information using the smart meter's session key
        session_key = bytes.fromhex(session_keys[smart_meter_id])
        cipher = AES.new(session_key, AES.MODE_EAX)
        nonce = cipher.nonce
        ciphertext, tag = cipher.encrypt_and_digest(
            json.dumps(
                {
                    "group_id": group_id,
                    "timestamp": get_timestamp(),
                    "x_a": x_a.hex(),
                    "group_key": group_key.hex(),
                }
            ).encode()
        )

        # Prepare the message
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": (nonce + tag + ciphertext).hex(),
        }

        # Send the message to the smart meter
        send_message(message)


def join_multicast_group(message, gateway_key):
    # Parse the received message
    smart_meter_id = message["smart_meter_id"]
    timestamp = message["timestamp"]
    encrypted_data = bytes.fromhex(message["encrypted_data"])

    # Extract the nonce, tag, and ciphertext from the encrypted data
    nonce = encrypted_data[:16]
    tag = encrypted_data[16:32]
    ciphertext = encrypted_data[32:]

    # Decrypt the multicast group information using the smart meter's session key
    session_key = bytes.fromhex(session_keys[smart_meter_id])
    cipher = AES.new(session_key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    multicast_info = json.loads(plaintext.decode())

    # Extract the multicast group information
    group_id = multicast_info["group_id"]
    x_a = bytes.fromhex(multicast_info["x_a"])
    group_key = bytes.fromhex(multicast_info["group_key"])

    # Prepare the response message
    response_message = {
        "smart_meter_id": smart_meter_id,
        "timestamp": get_timestamp(),
        "encrypted_data": ciphertext.hex(),
    }

    # Send the response message to the gateway
    send_message(response_message)

    # Store the multicast group information
    multicast_groups[group_id] = {"group_key": group_key, "members": [smart_meter_id]}
    print("Smart meter", smart_meter_id, "joined multicast group", group_id)


# Example usage
smart_meter_ids = ["SM1", "SM2", "SM3"]
serial_numbers = ["1234567890", "9876543210", "1357924680"]
gateway_key = b"abcdefghijklmnop"  # Example gateway key

# ... (Registration and authentication code remains the same) ...

multicast_groups = {}
sent_messages = []  # List to store sent messages
group_id = "GROUP1"
establish_multicast_group(group_id, ["SM1", "SM2"], gateway_key)

# Smart meters join the multicast group
for smart_meter_id in ["SM1", "SM2"]:
    # Retrieve the encrypted data from the message sent by the gateway
    encrypted_data = None
    for message in sent_messages:
        if message["smart_meter_id"] == smart_meter_id:
            encrypted_data = message["encrypted_data"]
            break

    if encrypted_data:
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": encrypted_data,
        }
        join_multicast_group(message, gateway_key)

Sending message: {"smart_meter_id": "SM1", "timestamp": 1714201174, "encrypted_data": "965ebb3f976c1d36da060bfc69ad17596f7487f3fea2222fe2cdb76f6b67424c1febb40d557a5df4e9549cbb977a134b0145eac8ccda5f414129a64f8912ee37ee9c0b79a78c83a564c25446ec1431d4eb5d8972b39fcb6c6e839fa585482d63f709c7ece6c4710f38e0f91d19f2b89b8470434927c16173c5882e94067f6a9bc4b3d30e4854d2b61bb2af53235048325efa3b48758f66713322b9370dc4fefd17021270f4038ce1ada5b4"}
Sending message: {"smart_meter_id": "SM2", "timestamp": 1714201174, "encrypted_data": "332499acbcb59bdb39df6900948ca88323f8dbe5941926968d33a03d0ed6c53680661554b5bc8127d2544ad5d2dd4ae00c5b7442605c62bac6f8cef2ab97e9c95ce99793e3f75bd170f64b8ef49130129b1b2e42faef2678c333d942ae850f5bb88e77335c4dea1ff4237daac6dfcda7b4835377f842d3347225325fc645fa57a01c20ab2df0a8c151a8941c3d0a01bec35d6a6d6586d4d4df1f3e4ab7989ff33c66816c51f64581e7b0d3"}
Sending message: {"smart_meter_id": "SM1", "timestamp": 1714201174, "encrypted_data": "1febb40d557a5df4e9549cbb977a134b0145eac8ccda5f414